In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#unpack dataset and organize into 80% train and 20% test in required folder structure
%cd /content
!mkdir images
!mkdir data
!mkdir TfRecordfiles
!mkdir unused_images

%cd /content/data
!mkdir train
!mkdir test
!tar -xf "/content/drive/My Drive/My Files/Project 2019 Data/images.tar"  -C /content/images

%cd /content/data/train
!mkdir masks
!mkdir images

%cd /content/data/test
!mkdir masks
!mkdir images

%cd /content/data/train/masks
!mkdir Lateral
!mkdir Primary
!mkdir Seeds
!mkdir Segmentation


%cd /content/data/test/masks
!mkdir Lateral
!mkdir Primary
!mkdir Seeds
!mkdir Segmentation

!cp -r /content/images/images/{0000..2140}.png /content/data/train/images
!cp -r /content/images/lateral16/{0000..2140}.png /content/data/train/masks/Lateral
!cp -r /content/images/primary16/{0000..2140}.png /content/data/train/masks/Primary
!cp -r /content/images/seeds/{0000..2140}.png /content/data/train/masks/Seeds
!cp -r /content/images/segmentation8/{0000..2140}.png /content/data/train/masks/Segmentation

!cp -r /content/images/images/{2141..2675}.png /content/data/test/images
!cp -r /content/images/lateral16/{2141..2675}.png /content/data/test/masks/Lateral
!cp -r /content/images/primary16/{2141..2675}.png /content/data/test/masks/Primary
!cp -r /content/images/seeds/{2141..2675}.png /content/data/test/masks/Seeds
!cp -r /content/images/segmentation8/{2141..2675}.png /content/data/test/masks/Segmentation
!cp -r /content/images/images/{2676..2695}.png /content/unused_images

/content
/content/data
/content/data/train
/content/data/test
/content/data/train/masks
/content/data/test/masks


In [0]:
import math
import os
import sys
import tensorflow as tf

slim = tf.contrib.slim

#State the labels filename
LABELS_FILENAME = ''
#===================================================  Dataset Utils  ===================================================
#functions for converter. Don't edit unless you know what you're doing. 

def int64_feature(values):
  """Returns a TF-Feature of int64s.

  Args:
    values: A scalar or list of values.

  Returns:
    a TF-Feature.
  """
  if not isinstance(values, (tuple, list)):
    values = [values]
  return tf.train.Feature(int64_list=tf.train.Int64List(value=values))


def bytes_feature(values):
  """Returns a TF-Feature of bytes.

  Args:
    values: A string.

  Returns:
    a TF-Feature.
  """
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))


def image_to_tfexample(image_data, image_format, height, width, class_id):
  return tf.train.Example(features=tf.train.Features(feature={
      'image/encoded': bytes_feature(image_data),
      'image/format': bytes_feature(image_format),
      'image/class/label': int64_feature(class_id),
      'image/height': int64_feature(height),
      'image/width': int64_feature(width),
  }))

def write_label_file(labels_to_class_names, dataset_dir,
                     filename=LABELS_FILENAME):
  """Writes a file with the list of class names.

  Args:
    labels_to_class_names: A map of (integer) labels to class names.
    dataset_dir: The directory in which the labels file should be written.
    filename: The filename where the class names are written.
  """
  labels_filename = os.path.join(dataset_dir, filename)
  with tf.gfile.Open(labels_filename, 'w') as f:
    for label in labels_to_class_names:
      class_name = labels_to_class_names[label]
      f.write('%d:%s\n' % (label, class_name))


def has_labels(dataset_dir, filename=LABELS_FILENAME):
  """Specifies whether or not the dataset directory contains a label map file.

  Args:
    dataset_dir: The directory in which the labels file is found.
    filename: The filename where the class names are written.

  Returns:
    `True` if the labels file exists and `False` otherwise.
  """
  return tf.gfile.Exists(os.path.join(dataset_dir, filename))


def read_label_file(dataset_dir, filename=LABELS_FILENAME):
  """Reads the labels file and returns a mapping from ID to class name.

  Args:
    dataset_dir: The directory in which the labels file is found.
    filename: The filename where the class names are written.

  Returns:
    A map from a label (integer) to class name.
  """
  labels_filename = os.path.join(dataset_dir, filename)
  with tf.gfile.Open(labels_filename, 'r') as f:
    lines = f.read().decode()
  lines = lines.split('\n')
  lines = filter(None, lines)

  labels_to_class_names = {}
  for line in lines:
    index = line.index(':')
    labels_to_class_names[int(line[:index])] = line[index+1:]
  return labels_to_class_names

#=======================================  Conversion Utils  ===================================================
def __init__(self):
#Create an image reader object for easy reading of the images
class ImageReader(object):
  """Helper class that provides TensorFlow image coding utilities."""

  
    # Initializes function that decodes RGB JPEG data.
    self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
    self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

  def read_image_dims(self, sess, image_data):
    image = self.decode_jpeg(sess, image_data)
    return image.shape[0], image.shape[1]

  def decode_jpeg(self, sess, image_data):
    image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
    assert len(image.shape) == 3
    assert image.shape[2] == 3
    return image


def _get_filenames_and_classes(dataset_dir):
  """Returns a list of filenames and inferred class names.

  Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.

  Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
  """
  # print 'DATASET DIR:', dataset_dir
  # print 'subdir:', [name for name in os.listdir(dataset_dir)]
  # dataset_main_folder_list = []
  # for name in os.listdir(dataset_dir):
  # 	if os.path.isdir(name):
  # 		dataset_main_folder_list.append(name)
  dataset_main_folder_list = [name for name in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir,name))]
  dataset_root = os.path.join(dataset_dir, dataset_main_folder_list[0])
  directories = ["/content/data/test/images"]
  class_names = ['Primary','Lateral','Seeds','Segmentation']
  for filename in os.listdir(dataset_root):
    path = os.path.join(dataset_root, filename)
    if os.path.isdir(path):
      directories.append(path)
      class_names.append(filename)

  photo_filenames = []
  for directory in directories:
    for filename in os.listdir(directory):
      path = os.path.join(directory, filename)
      photo_filenames.append(path)

  return photo_filenames, sorted(class_names)


def _get_dataset_filename(dataset_dir, split_name, shard_id, tfrecord_filename, _NUM_SHARDS):
  output_filename = '%s_%s_%05d-of-%05d.tfrecord' % (
      tfrecord_filename, split_name, shard_id, _NUM_SHARDS)
  return os.path.join(dataset_dir, output_filename)


def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir, tfrecord_filename, _NUM_SHARDS):
  """Converts the given filenames to a TFRecord dataset.

  Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
  """
  assert split_name in ['train', 'validation']

  num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

  with tf.Graph().as_default():
    image_reader = ImageReader()

    with tf.Session('') as sess:

      for shard_id in range(_NUM_SHARDS):
        output_filename = _get_dataset_filename(
            dataset_dir, split_name, shard_id, tfrecord_filename = tfrecord_filename, _NUM_SHARDS = _NUM_SHARDS)

        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
          start_ndx = shard_id * num_per_shard
          end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
          for i in range(start_ndx, end_ndx):
            sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
                i+1, len(filenames), shard_id))
            sys.stdout.flush()

            # Read the filename:
            image_data = tf.gfile.FastGFile(filenames[i], 'r').read()
            height, width = image_reader.read_image_dims(sess, image_data)

            class_name = os.path.basename(os.path.dirname(filenames[i]))
            class_id = class_names_to_ids[class_name]

            example = image_to_tfexample(
                image_data, 'jpg', height, width, class_id)
            tfrecord_writer.write(example.SerializeToString())

  sys.stdout.write('\n')
  sys.stdout.flush()

def _dataset_exists(dataset_dir, _NUM_SHARDS, output_filename):
  for split_name in ['train', 'validation']:
    for shard_id in range(_NUM_SHARDS):
      tfrecord_filename = _get_dataset_filename(
          dataset_dir, split_name, shard_id, output_filename, _NUM_SHARDS)
      if not tf.gfile.Exists(tfrecord_filename):
        return False
  return True


In [0]:
%cd /content
!git clone https://github.com/I-Alpha/Mask_RCNN.git

/content
Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.90 MiB | 43.87 MiB/s, done.
Resolving deltas: 100% (571/571), done.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os


#construct list for image paths and their corresponding masks in the same order
#image_path = "/content/data/images/train/*.png"
dta_type = "test" #specifies train or test folder to work on. 

image_path = []
for filename in os.listdir("/content/data/"+dta_type+"/images"):
    image_path.append("/content/data/"+dta_type+"/images/"+filename)

Pmask_path = []
for i, filename in enumerate(image_path):
    Pmask_path.append("/content/data/" + dta_type + "/masks/Primary/" + os.path.basename(filename))
    
Lmask_path = []
for i, filename in enumerate(image_path):
    Lmask_path.append("/content/data/" + dta_type + "/masks/Lateral/" + os.path.basename(filename))

Smask_path = []
for i, filename in enumerate(image_path):
    Smask_path.append("/content/data/" + dta_type + "/masks/Seeds/" + os.path.basename(filename))

Segmask_path = []
for i, filename in enumerate(image_path):
    Segmask_path.append("/content/data/" + dta_type + "/masks/Segmentation/" + os.path.basename(filename))


In [0]:
from PIL import Image
# path to where the tfrecords will be stored (change to your customized path).
# I can't run this here in Kaggle because of permission. 
# Any comment on how to write temporary files in Kaggle will be appreciated.
tfrecords_filename = "/content/TfRecordfiles/"+ dta_type + "ing_data.tfrecords"
#get a writer for the tfrecord file.
writer = tf.python_io.TFRecordWriter(tfrecords_filename)
#write data/masks into tfrecords
for i in range(len(image_path)):
    print(i+1,len(image_path)-1)
    img = Image.open(image_path[i]).convert('RGB')
    img = np.array(img)
    
    Pmask = np.array(mpimg.imread(Pmask_path[i]),np.dtype(np.uint8)) 
    Lmask = np.array(mpimg.imread(Lmask_path[i]),np.dtype(np.uint8))
   #Smask = np.array(mpimg.imread(Smask_path[i]),np.dtype(np.uint8))   Smasks are 24-bit.Have to convert.
    Smask = Image.open(Smask_path[i]).convert('L')
    Smask = np.array(Smask,np.dtype(np.uint8))
    #print((Smask))
    Segmask = np.array(mpimg.imread(Segmask_path[i]),np.dtype(np.uint8))
    
    
    height = img.shape[0]
    width = img.shape[1]
    img_raw = img.tostring()
    Pmask_raw = Pmask.tostring()
    Lmask_raw = Lmask.tostring()
    Smask_raw = Smask.tostring()
    Segmask_raw = Segmask.tostring()
    
    #save the heights and widths as well so, which 
    #are needed when decoding from tfrecords back to images
    example = tf.train.Example(features=tf.train.Features(feature={
                                                          'height': _int64_feature(height),
                                                          'width': _int64_feature(width),
                                                          'image_raw':bytes_feature(img_raw),
                                                          'Pmask_raw': _bytes_feature(Pmask_raw),
                                                          'Lmask_raw': _bytes_feature(Lmask_raw),
                                                          'Smask_raw': _bytes_feature(Smask_raw),
                                                          'Segmask_raw': _bytes_feature(Segmask_raw)}
                                                          ))
    writer.write(example.SerializeToString())
writer.close()


1 534


NameError: ignored

In [0]:
#run the following to verify the created tfrecord file.
record_iterator = tf.io.tf_record_iterator(tfrecords_filename)
xcount = 1
for string_record in record_iterator:
    
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    #get features 
    height = int(example.features.feature['height'].int64_list.value[0])
    width = int(example.features.feature['width'].int64_list.value[0])
    img_string = (example.features.feature['image_raw'].bytes_list.value[0])
    Pmask_string = (example.features.feature['Pmask_raw'].bytes_list.value[0])
    Lmask_string = (example.features.feature['Lmask_raw'].bytes_list.value[0])
    Smask_string = (example.features.feature['Smask_raw'].bytes_list.value[0])
    Segmask_string = (example.features.feature['Segmask_raw'].bytes_list.value[0])
    
    #get 1d array from string of features
    img_1d = np.frombuffer(img_string, dtype=np.uint8)
    Pmask_1d = np.frombuffer(Pmask_string, dtype=np.uint8)
    Lmask_1d = np.frombuffer(Lmask_string, dtype=np.uint8)
    Smask_1d = np.frombuffer(Smask_string, dtype=np.uint8)
    Segmask_1d = np.frombuffer(Segmask_string, dtype=np.uint8)

    #reshape back to their original shape from a 1D array read from tfrecords
    img = img_1d.reshape((height, width, -1))
    Pmask = Pmask_1d.reshape((height, width))
    Lmask = Lmask_1d.reshape((height, width))
    Smask = Smask_1d.reshape((height, width))
    Segmask = Segmask_1d.reshape((height, width))
    
  
    print("image : ",xcount)
    xcount+=1
    plt.imshow(img)
    plt.show()
    plt.imshow(Pmask)
    plt.show()
    plt.imshow(Lmask)
    plt.show()
    plt.imshow(Smask)
    plt.show()
    plt.imshow(Segmask)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
%cd /content 
!git clone https://github.com/matterport/Mask_RCNN.git
/content/drive/My Drive/My Files/Project 2019 Data/Labels

In [0]:


Labels_map = {0: {'id': 0, 'name': 'Primary'},
              1: {'id': 1, 'name': 'Lateral'},
              2: {'id': 2, 'name': 'Seeds'},
              3: {'id': 3, 'name': 'Segmentation'}}

train_dataset = tf.data.TFRecordDataset("/content/TfRecordfiles/testing_data.tfrecords")
test_dataset = tf.data.TFRecordDataset("/content/TfRecordfiles/training_data.tfrecords")